In [145]:
import requests
import pandas as pd

In [61]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.amazon.in/')

In [63]:
search_button = driver.find_element_by_id('twotabsearchtextbox')
search_button.click()
search_button.send_keys("analog watches for men")
search_button.submit()

Analysing the automated chrome driver, we can observe that there are 7 total number of pages.

Collecting all the URLs (for each page).

In [71]:
all_pages_url = []
for i in range(1,8):
    all_pages_url.append('https://www.amazon.in/s?k=analog+watches+for+men&page='+str(i)+'&ref=sr_pg_7')
    

In [74]:
# creating header files for making http requests
HEADERS = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
base_url = 'https://www.amazon.in/'


# Iterating over each page and finding the link of each watch within the page,
# Further iterating over each watch link and extracting the desired data

In [140]:
all_watches = []                #contains the data of each watch within 7 pages

In [141]:
for page in all_pages_url:
    response = requests.get(page,headers=HEADERS)
    soup = BeautifulSoup(response.text)
    
    # creating a list of all watches on the current page (links)
    curr_watches = []
    for i in soup.find_all('a',target = "_blank",class_ = 'a-link-normal s-underline-text s-underline-link-text s-link-style'):
        curr_watches.append(base_url+i.attrs['href'])
    
    # extracting the details of each watch
    for i in curr_watches:
        try:    
            response1 = requests.get(i,headers=HEADERS)
            soup1 = BeautifulSoup(response1.text,'html.parser')
            if str(response1.status_code).startswith('2'):

                title = soup1.find(id='title').text.strip()
                availability = soup1.find(id='availability').text.strip()
                review_counts = int(soup1.find(id='acrCustomerReviewText').text.split()[0].replace(',',''))
                ratings = float(soup1.find('span',class_='a-size-base a-color-base').text.strip())
                price = float(soup1.find('span',class_='a-price-whole').text.replace(",",''))

            all_watches.append([title,review_counts,ratings,availability,price])
            
        except:
            continue

# as the data is now extracted, we can convert the data to Data Frames

In [166]:
import pandas as pd
df = pd.DataFrame(all_watches,columns=['Title','Reviews',"Ratings","Availability",'Price'])

Ceating brand feature for each watch 

In [167]:
def brand(s):
    return s.split()[0]

df['Brand'] = df.Title.apply(brand)

In [168]:
# re-ordering df
df = df[['Title',"Brand",'Reviews',"Ratings","Availability",'Price']]

In [169]:
# filling the empty values in Availability wihth 'Out of stock'
def empty_availability(s):
    if s == '':
        return 'Out of stock'
    else:
        return s
    
df.Availability = df.Availability.apply(empty_availability)

In [170]:
# Finally exporting the data as .csv

In [175]:
df.to_csv("raw_data.csv",index=False)